<a href="https://colab.research.google.com/github/francesita/ProfnerTask7a/blob/main/Profner_mBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model for Profner Dataset Task 7

Plan


1.   Build baseline
2.   Build model using BERT



In [1]:
#mount googleDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

##Load datasets from csv files

We load the data from csv files (it has already been separated, so we will not divide data between train, val, test)

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/train.csv')
val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val.csv')

In [ ]:
#loading english data--- to augment positive examples

In [ ]:
print('amount of augmented data', len(aug_data_es))

amount of augmented data 865


In [ ]:
train = train_data.tweet
aug =  aug_data_es.tweet
val = val_data.tweet

labels = train_data.label[:100]
aug_labels = aug_data_es.label[:100]
val_labels = val_data.label

In [ ]:
type(aug[0])
aug[0]

'denuncian a un internista que cuestionó si vale la pena salvar rojos del coronavirus'

In [ ]:
print(len(train), len(labels))
print(len(aug), len(aug_labels))
print(len(val), len(val_labels))

100 100
100 100
2000 2000


In [ ]:
del train_data
del val_data

In [ ]:
#concatenate traina nd augmented (for train) data
train.append(aug)
labels_aug = labels.append(aug_labels)

In [ ]:
print(train, " ", labels)
print(len(train_aug))
print(len(labels_aug))
type(labels[0])



0     ['CHINA: libera una pandemia\n', 'EE.UU: liber...
1     ['San Francisco (EEUU) hace 100 años tras vivi...
2     ['Porfi poneos la mascarilla o tendremos 28 nu...
3     ['El nuevo „covid normas y reglas recibimiento...
4     ['Si el confinamiento ha dejado algo tocada tu...
                            ...                        
95    ['@DGobiernoMadrid Gente que no sabe hasta cuá...
96    ['Hoy a las 20h, concentración en tu centro de...
97    ['Coronavirus | Bruselas da un toque a España ...
98    ['Quienes no consideran necesaria la mascarill...
99    ['Hoy damos las gracias a todo el personal san...
Name: tweet, Length: 100, dtype: object   0     0
1     0
2     0
3     0
4     0
     ..
95    0
96    0
97    0
98    0
99    1
Name: label, Length: 100, dtype: int64
200
200


numpy.int64

In [ ]:
#Shuffle data
idx = np.random.permutation(len(train))
train, y_train = train[idx], labels[idx]

##Preprocess tweets 

1. remove twitter handles, links, hashtags, punctuation etc. so that we are left solely with text in the tweet
2. tokenize tweets

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy, Recall
import tensorflow.keras.backend as K

In [4]:
import re

def preprocess_tweets(tweet):
  """
  - remove hashtags, twitter handles, url's
  """
  #might want to try replacing @mentions with a word, such as person/persona?
  #tweet = re.findall(r'#(\w+)', tweet, re.UNICODE)
  tweet = ' '.join(re.sub(r"(@[A-Za-z0-9_]+)|([áéíóúñü][^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())
  tweet = tweet.replace("#", "").replace("_", " ")
  #remove emojis from tweets
  #tweet = remove_emoji(tweet)
  return tweet

In [5]:
import string
def remove_punctuation(tknzd_tweet):
  spanish_pnct = (['¿','¡',':','¨','...',"'",'€','£','$','"','@'])
  cln_twt = []
  for token in tknzd_tweet:
    if token not in string.punctuation and token not in spanish_pnct:
      cln_twt.append(token)
  
  return cln_twt

In [6]:
!pip install demoji
import demoji
demoji.download_codes()
#remove emojis from tweet
def remove_emoji(tweet):
  tweet = demoji.replace(tweet, "")

  return tweet

... OK (Got response in 0.16 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [ ]:
text = 'Cuñado, vén conmigo mañana para comer arroz. ¡Tú eres tán estúpida! @lal_oca #ror_dfa 😜🇵🇷💀⛵🈵🇸🇻 @'
preprocess_tweets(text)

'Cuñado, vén conmigo mañana para comer arroz. ¡Tú eres tán estúpida! ror dfa  @'

We can check that our data is properly encoded doing the following

#Model using mBert Sequence Classifier


Load data for the model. The data needs to be encoded differently from the BiLSTM classifier above. 

In [7]:
#import data
train_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/train.csv')
val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val.csv')

In [8]:

test_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/test.csv')

In [9]:
test = test_data.tweet

In [10]:
es_train = list(train_data.tweet)
#eng_train = list(eng_data.tweet)
val = val_data.tweet

es_labels = list(train_data.label)
#eng_labels = list(eng_data.label) 
val_labels = val_data.label

In [11]:
#turn train data lists to numpy arrays
train = np.asarray(es_train)
#eng_train = np.asarray(eng_train)

labels = np.asarray(es_labels)
#eng_labels= np.asarray(eng_labels)

#Combine training data
#train = np.append(es_train, eng_train)
#labels = np.append(es_labels, eng_labels)

In [ ]:
len(train)
print(labels[0])

0


In [12]:
#shuffle data for both train and val sets
idx = np.random.permutation(len(train))
x_train, y_train = train[idx],labels[idx]

x_val, y_val = val,val_labels

In [ ]:
print(len(x_train))

6000


##Text processing for mBERT model

In [13]:
!pip install transformers

     |████████████████████████████████| 1.9MB 17.7MB/s 
     |████████████████████████████████| 890kB 55.0MB/s 
     |████████████████████████████████| 3.2MB 55.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=4a02a2d87e297da9a63e8acceb632a5232a811f4941547d97b4a08c3799a5d5b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


##Preprocess tweet for mBERT


In [14]:
!pip install transformers
from transformers import BertTokenizer, TFBertForSequenceClassification

In [ ]:
#load mBERT tokenizer, we will use it later in a function (this one is used to encode to find max_len)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case = True)

In [ ]:
tokenizer.encode("Ayer me cague en la puta madre de este trabajo")

[101,
 11612,
 10177,
 10525,
 10678,
 17734,
 10109,
 10106,
 28721,
 15452,
 10102,
 10494,
 15858,
 102]

In [17]:
#defining function to preprocess with Bert
def preprocess_bert(tweets, pt_tokenizer_model, max_len):
    '''
    This function will do a variety of things to prep data for Roberta (through encode_plus) which includes:
      -tokenizing tweet
      -adding <s> BOS token (used for classification) and </s></s> as a [SEP] token to start and end of tweet
      -pad or truncate the tweet to max length
      -map tokens to their encoding or id
      -creates attention mask: this is a mask used for attention when a batch has varying length of sentences
      -returns a dict of outputs
      -all this comes from fine-tuning tutorial from skimai.com
    '''

    tokenizer = BertTokenizer.from_pretrained(pt_tokenizer_model, do_lower_case = True)

    encoded_tweets = []
    #attention masks indicated to the model we will use, which tokens should be attended to, example indicates position of padding
    #so model should not pay attention to these. 
    attention_masks = []


    for tweet in tweets:
        encoded_tweet = tokenizer.encode_plus(
            text = preprocess_tweets(tweet),
            add_special_tokens=True, #cls and sep tokens 
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True)

        #add outputs to list
        encoded_tweets.append(encoded_tweet.get('input_ids'))
        attention_masks.append(encoded_tweet.get('attention_mask'))
      
    #we now convert the lists to tensors
    encoded_tweets = np.asarray(encoded_tweets, dtype='int32')
    attention_masks = np.asarray(attention_masks, dtype='int32')

    return encoded_tweets, attention_masks



In [ ]:
#Now we find maximum length in our list of tweets with the special tokens

#we concatenate training and validation sets 
all_tweets = np.concatenate([train_data.tweet, val_data.tweet])

#encode using tokenizer function (not our fucntion with encoded_plus, this is because we want to see max_length of tweet with special tokens)
all_tweets_encoded = [tokenizer.encode(tweet, add_special_tokens=True) for tweet in all_tweets]
print(all_tweets_encoded[0])
print(tokenizer.decode(all_tweets_encoded[0]))

#find maximum length
max_len = max([len(tweet) for tweet in all_tweets_encoded])
print('max length of tweets:', max_len)

#find avg length
# we might want to use avg len rather than max length in future
avg_len = int(np.average([len(tweet) for tweet in all_tweets_encoded]))
print('average len:' ,avg_len)

In [18]:
print(type(x_train))
print(x_train[0])

<class 'numpy.ndarray'>
['🔝#SoniaPérez: "Podemos conseguir que el #COVID19 no corra tanto quedándonos en casa, pero para que eso sea posible algunos tienen que salir de la suya"\n', '\n', '🗞️Artículo en @elcorreo_com  de la consejera socialista de Turismo, Comercio y Consumo del Gobierno Vasco \n', '\n', '#FuertesYUnidos https://t.co/yuCdjWTjnI']


In [19]:
#define var with model type
m_tokenizer = 'bert-base-multilingual-uncased'
max_len = 80
#tokenize our tweets using the pre-proprocess function we defined earlier
train_inputs, train_masks = preprocess_bert(x_train, m_tokenizer, max_len)
val_inputs, val_masks = preprocess_bert(x_val, m_tokenizer, max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [31]:
#tokenize and preprocess test
m_tokenizer = 'bert-base-multilingual-uncased'
test_inputs, test_masks = preprocess_bert(test, m_tokenizer, max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(train_inputs[0])
print(train_masks[0])
print(y_train[0])

[  101   138   112 10265 16422 10107 10128 23145 41881 11823 10119 12715
   110 10109 10117 29346 10102 10106 84315 10102 20241 70294   117 10173
 11589   119 10292 46720 38183 10321 10109 10426 17530 11310 10190   112
   140   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
0


In [ ]:
print('shape of test_inputs:', test_inputs.shape)
print('shape of test_masks', test_masks.shape)

shape of test_inputs: (27000, 80)
shape of test_masks (27000, 80)


In [ ]:
print('shape of train_inputs:', train_inputs.shape)
print('shape of train_masks', train_masks.shape)

##Fine-tune mBERT classification model

In [20]:
import tensorflow as tf

#making labels into numpy arrays for training in transformers and keras
def create_transformer_model(pre_trained_model, num_labels, max_len):
  transformer_model = TFBertForSequenceClassification.from_pretrained(pre_trained_model, num_labels=num_labels)

  #We create keras tensors
  input_ids = tf.keras.layers.Input(shape=(max_len,), name='train_input', dtype=tf.int32)
  input_masks = tf.keras.layers.Input(shape=(max_len), name='train_masks', dtype=tf.int32)

  #take into account single dimension
  seq_outputs = transformer_model(input_ids, input_masks)[0]
  outputs = tf.keras.layers.Dense(num_labels, activation='sigmoid')(seq_outputs)

  model = tf.keras.models.Model(inputs=[input_ids,input_masks], outputs=outputs)

  print(model.summary())

  return model

In [63]:
pt_trans_model = 'bert-base-multilingual-uncased'
num_labels = 1
loss = 'binary_crossentropy'
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)


model = create_transformer_model(pt_trans_model, num_labels, 80)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
train_input (InputLayer)        [(None, 80)]         0                                            
__________________________________________________________________________________________________
train_masks (InputLayer)        [(None, 80)]         0                                            
__________________________________________________________________________________________________
tf_bert_for_sequence_classifica TFSequenceClassifier 167357185   train_input[0][0]                
                                                                 train_masks[0][0]                
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 1)            2           tf_bert_for_sequence_classi

In [64]:
#make labels into numpy arrays, and prep x_val
x_val = [val_inputs, val_masks]
#turn labels into tensors
y_train = np.asarray(y_train, dtype='int32')
y_val = np.asarray(y_val, dtype='int32')

In [23]:
!nvidia-smi -L

device_name = tf.test.gpu_device_name()
print(device_name)

GPU 0: Tesla T4 (UUID: GPU-927f2c9a-a68c-2ff6-4af2-ada1a4bda9d7)
/device:GPU:0


In [27]:
#fit model to input data
#class_weight = {0: 1.,
#                1: 3.}
with tf.device('/device:GPU:0'):
  model.fit([train_inputs, train_masks], y_train, validation_data=(x_val, y_val), epochs=3, batch_size = 32)


Epoch 1/3
188/188 [==============================] - 120s 588ms/step - loss: 0.5526 - accuracy: 0.7494 - val_loss: 0.2940 - val_accuracy: 0.9035
Epoch 2/3
188/188 [==============================] - 112s 595ms/step - loss: 0.2637 - accuracy: 0.9073 - val_loss: 0.1859 - val_accuracy: 0.9420
Epoch 3/3
188/188 [==============================] - 112s 594ms/step - loss: 0.1448 - accuracy: 0.9575 - val_loss: 0.1675 - val_accuracy: 0.9480


In [33]:
#save model
model.save_weights('/content/drive/MyDrive/Colab/saved_model/mBert_profner_emoji.hdf5')

##Testing and Error Analysis on mBert Classifier

In [65]:
#load model if not already
model.load_weights('/content/drive/MyDrive/Colab/saved_model/mBert_profner3.hdf5')
#del es_model
#es_model = tf.saved_model.load('/content/drive/MyDrive/Colab/saved_model/mBert_profner3')


In [28]:
def assign_class(a_pred):
  pred = []
  for i in range(len(a_pred)):
    if a_pred[i] >= 0.5:
      pred.append(1)
    else:
      pred.append(0)
  return pred

In [66]:
#make predictions on model
y_pred = model.predict([val_inputs, val_masks])
es_pred = assign_class(y_pred)


In [ ]:
print(y_pred[:10])
print(es_pred[:10])

[[0.00605112]
 [0.00700646]
 [0.9257546 ]
 [0.043198  ]
 [0.00858568]
 [0.01304464]
 [0.00830286]
 [0.01447904]
 [0.9908717 ]
 [0.00924681]]
[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]


In [30]:
#see what is going wrong
print(es_pred[:20])
print(type(val))
es_pred_dic = {} #index of tweet is key and values are predicted value
for i, p in enumerate(es_pred):
  if y_val[i] != p:
    es_pred_dic[i] = {}
    es_pred_dic[i]['tweet'] = val[i]
    es_pred_dic[i]['pred'] = p
    es_pred_dic[i]['true'] = y_val[i]

[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]
<class 'pandas.core.series.Series'>


In [31]:
len(es_pred_dic.items())

104

###Confusion Matrix for mBert Model

In [67]:
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import classification_report

y_true = y_val
y_pred = es_pred

print(confusion_matrix(y_true, y_pred, labels=[1,0]))
print(classification_report(y_true, y_pred, target_names=["non-prof", "prof"]))
print('f1-score:', f1_score(y_true, y_pred))

[[ 401   76]
 [  28 1495]]
              precision    recall  f1-score   support

    non-prof       0.95      0.98      0.97      1523
        prof       0.93      0.84      0.89       477

    accuracy                           0.95      2000
   macro avg       0.94      0.91      0.93      2000
weighted avg       0.95      0.95      0.95      2000

f1-score: 0.8852097130242826


# New section

#Model using Bert Uncased Classifier
We train a model using bert uncased using the English translation of the tweets 

In [ ]:
#load english tweet data
train_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/train-eng.csv')
val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val-eng.csv')

In [ ]:
eng_train = train_data.tweet
eng_val = val_data.tweet

labels = train_data.label
val_labels = val_data.label

In [ ]:
#shuffle data for both train and val sets
idx = np.random.permutation(len(eng_train))
x_train, y_train = eng_train[idx],labels[idx]

x_val, y_val = eng_val, val_labels

In [ ]:
print(x_val[0])

[[  101   138   112 ...     0     0     0]
 [  101   138   112 ...     0     0     0]
 [  101   138   112 ...     0     0     0]
 ...
 [  101   138   112 ...   136 76320   102]
 [  101   138   112 ...     0     0     0]
 [  101   138   112 ...     0     0     0]]


##Preprocess tweet for Bert
- we use a function preprocess_bert defined earlier in the code the encode our tweets, except we encode tweets to the pretrained bert-uncased model

In [ ]:
#define var with model type
eng_tokenizer = 'bert-base-uncased'
max_len = 80
#preprocess using bert
eng_train_input, eng_train_masks = preprocess_bert(x_train, eng_tokenizer, max_len) 
eng_val_input, eng_val_masks = preprocess_bert(x_val, eng_tokenizer, max_len) 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

##Fine-Tune Bert uncased classification model
We use the create_model function defined in the mBert section. We will provide the transformer model type (bert-uncased in our case), the number of classes in our problem, and max_len of each tweet

In [ ]:
import tensorflow as tf

#making labels into numpy arrays for training in transformers and keras
def create_eng_transformer_model(pre_trained_model, num_labels, max_len):
  transformer_model = TFBertForSequenceClassification.from_pretrained(pre_trained_model, num_labels=num_labels)

  #We create keras tensors
  input_ids = tf.keras.layers.Input(shape=(max_len,), name='train_input', dtype=tf.int32)
  input_masks = tf.keras.layers.Input(shape=(max_len), name='train_masks', dtype=tf.int32)

  #take into account single dimension
  seq_outputs = transformer_model(input_ids, input_masks)[0]
  #dense_layer = tf.keras.layers.Dense(64, activation='tanh')(seq_outputs)
  outputs = tf.keras.layers.Dense(num_labels, activation='sigmoid')(seq_outputs)
  
  model = tf.keras.models.Model(inputs=[input_ids,input_masks], outputs=outputs)

  print(model.summary())

  return model

In [ ]:
pt_trans_model = 'bert-base-uncased'
num_labels = 1
loss = 'binary_crossentropy'
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)


eng_model = create_eng_transformer_model(pt_trans_model, num_labels, 80)
eng_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
train_input (InputLayer)        [(None, 80)]         0                                            
__________________________________________________________________________________________________
train_masks (InputLayer)        [(None, 80)]         0                                            
__________________________________________________________________________________________________
tf_bert_for_sequence_classifica TFSequenceClassifier 109483009   train_input[0][0]                
                                                                 train_masks[0][0]                
__________________________________________________________________________________________________
dense_20 (Dense)                (None, 1)            2           tf_bert_for_sequence_class

In [ ]:
#make labels into numpy arrays, and prep x_val
x_val = [eng_val_input, eng_val_masks]
#turn labels into tensors
y_train = np.asarray(y_train, dtype='int32')
y_val = np.asarray(y_val, dtype='int32')

In [ ]:
#fit model to input data
class_weight = {0: 1.,
                1: 17.}
with tf.device('/device:GPU:0'):
  eng_model.fit([eng_train_input, eng_train_masks], y_train, validation_data=(x_val, y_val), epochs=3, batch_size = 32, class_weight=class_weight)

Epoch 1/3
188/188 [==============================] - 67s 359ms/step - loss: 0.7222 - accuracy: 0.8925 - val_loss: 0.3186 - val_accuracy: 0.9255
Epoch 2/3
188/188 [==============================] - 66s 353ms/step - loss: 0.4242 - accuracy: 0.9357 - val_loss: 0.2429 - val_accuracy: 0.9255
Epoch 3/3
188/188 [==============================] - 66s 353ms/step - loss: 0.2784 - accuracy: 0.9565 - val_loss: 0.3016 - val_accuracy: 0.9215


In [ ]:
eng_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
train_input (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
train_masks (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_bert_for_sequence_classifica TFSequenceClassifier 109483009   train_input[0][0]                
                                                                 train_masks[0][0]                
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 1)            2           tf_bert_for_sequence_classi

In [ ]:
eng_model.save_weights('/content/drive/MyDrive/Colab/saved_model/engBert_profner_weight_2.hdf5')

##Testing and Error Analysis on Bert Classifier


In [ ]:
#import model if needed
#eng_model.load_weights('/content/drive/MyDrive/Colab/saved_model/engBert_profner3.hdf5')

In [ ]:
#make predictions on model
eng_y_pred = eng_model.predict(x_val)
eng_pred = assign_class(eng_y_pred)

In [ ]:
#see what is going wrong
print(eng_pred[:20])
print(type(val))
eng_pred_dic = {} #index of tweet is key and values are predicted value
for i, p in enumerate(eng_pred):
  if y_val[i] != p:
    eng_pred_dic[i] = {}
    eng_pred_dic[i]['tweet'] = eng_val[i]
    eng_pred_dic[i]['pred'] = p
    eng_pred_dic[i]['true'] = y_val[i]

[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]
<class 'pandas.core.series.Series'>


In [ ]:
print('incorrect classification in english model', len(eng_pred_dic.items()))
print(eng_pred_dic)

incorrect classification in english model 157
{6: {'tweet': '[\'"The initial mismanagement of #COVID19 has not been a failure of U.S. intelligence (which did warn policymakers), but has been one of the worst and most dangerous policy failures in U.S. history."\\n\', \'https://t.co/mkpU8YENcz\']', 'pred': 1, 'true': 0}, 21: {'tweet': "['🚑In Santander, an ambulance wasting precious time because of the far-right demonstration.\\n', '\\n', 'It serves as a metaphor for what certain irresponsible attitudes are implying in the midst of a pandemic, while working people are organizing to save lives.\\n', '#ElVirusSoisVoxotros https://t.co/7l2v2wfoJm']", 'pred': 1, 'true': 0}, 41: {'tweet': "['Today we have approved in the Council of Ministers the #COVID19 Fund for the Autonomous Communities. Endowed with 16,000 million euros, it is the largest disbursement of resources for the autonomous regions in the recent history of our democracy, reflecting our firm commitment to the autonomous State. http

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score
from sklearn.metrics import classification_report

y_true = y_val
y_pred = eng_pred

print(confusion_matrix(y_true, eng_pred, labels=[1,0]))
print(classification_report(y_true, eng_pred, target_names=["non-prof", "prof"]))
print('f1-score:', f1_score(y_true, y_pred))

[[ 441   36]
 [ 121 1402]]
              precision    recall  f1-score   support

    non-prof       0.97      0.92      0.95      1523
        prof       0.78      0.92      0.85       477

    accuracy                           0.92      2000
   macro avg       0.88      0.92      0.90      2000
weighted avg       0.93      0.92      0.92      2000

f1-score: 0.848893166506256


#Combine Models for Classification Task
We combine use the Spanish model to classify the tweets, then the English model to classify tweets translated into English. 
We add the output, and classify as 1 (Prof) if sum of outputs is greater than ..... else, class as 0 (non-Prof)

In [ ]:
#Checking amount of tweets that got miscalssified in both models: Analysis first
missed_idx_eng = np.array(list(eng_pred_dic.keys()))
missed_idx_es = np.array(list(es_pred_dic.keys()))

print(type(list(eng_pred_dic.items())))
common_idx_missed = np.intersect1d(missed_idx_eng, missed_idx_es)
count_overlap = len(common_idx_missed)


print('Common values between preds in both languages:')
print(common_idx_missed)

print('Overlap count:', count_overlap)

unique_incorrect = (len(missed_idx_eng) - 67)
print(unique_incorrect)

<class 'list'>
Common values between preds in both languages:
[   6   41   53   57  143  174  181  243  262  282  288  328  352  378
  463  499  551  620  651  782  790  819  831  838  900  932  989 1003
 1006 1083 1153 1159 1197 1226 1271 1277 1297 1300 1406 1439 1447 1478
 1487 1508 1512 1570 1595 1629 1642 1669 1680 1700 1764 1791 1809 1810
 1824 1826 1843 1856 1879 1910 1918 1921 1932 1997]
Overlap count: 66
48


##Import models for Majority Vote Method
We import two models in Spanish and one model in English, we will compare the predicted outputs of the three models and pick the majority vote "class" as our final pred

In [ ]:
#define variable for model creation
es_pt_model = 'bert-base-multilingual-uncased'
eng_pt_model = 'bert-base-uncased'
num_labels = 1
loss = 'binary_crossentropy'
max_len = 70
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

#Compile models
eng_model = create_eng_transformer_model(eng_pt_model, num_labels, max_len)
eng_model2 = create_eng_transformer_model(eng_pt_model, num_labels, max_len)
#es_model1 = create_transformer_model(es_pt_model, num_labels, max_len)
es_model2 = create_transformer_model(es_pt_model, num_labels, max_len)

eng_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
eng_model2.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
#es_model1.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
es_model2.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

#we load both models (if needed)
#es_model1.load_weights('/content/drive/MyDrive/Colab/saved_model/mBert_profner.h5')
es_model2.load_weights('/content/drive/MyDrive/Colab/saved_model/mBert_profner1.hdf5')
eng_model.load_weights('/content/drive/MyDrive/Colab/saved_model/engBert_profner.hdf5')
eng_model2.load_weights('/content/drive/MyDrive/Colab/saved_model/engBert_profner2.hdf5')


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
train_input (InputLayer)        [(None, 70)]         0                                            
__________________________________________________________________________________________________
train_masks (InputLayer)        [(None, 70)]         0                                            
__________________________________________________________________________________________________
tf_bert_for_sequence_classifica TFSequenceClassifier 109483009   train_input[0][0]                
                                                                 train_masks[0][0]                
__________________________________________________________________________________________________
dense_18 (Dense)                (None, 1)            2           tf_bert_for_sequence_class

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
train_input (InputLayer)        [(None, 70)]         0                                            
__________________________________________________________________________________________________
train_masks (InputLayer)        [(None, 70)]         0                                            
__________________________________________________________________________________________________
tf_bert_for_sequence_classifica TFSequenceClassifier 109483009   train_input[0][0]                
                                                                 train_masks[0][0]                
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 1)            2           tf_bert_for_sequence_class

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
train_input (InputLayer)        [(None, 70)]         0                                            
__________________________________________________________________________________________________
train_masks (InputLayer)        [(None, 70)]         0                                            
__________________________________________________________________________________________________
tf_bert_for_sequence_classifica TFSequenceClassifier 167357185   train_input[0][0]                
                                                                 train_masks[0][0]                
__________________________________________________________________________________________________
dense_20 (Dense)                (None, 1)            2           tf_bert_for_sequence_class

In [ ]:
#Import data for English and Spanish tweets
eng_val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val-eng.csv')
es_val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val.csv')

In [ ]:
eng_val = eng_val_data.tweet
es_val = es_val_data.tweet

labels = eng_val_data.label

##Preprocess val/test (if not already)

In [ ]:
eng_tokenizer = 'bert-base-uncased'
es_tokenizer = 'bert-base-multilingual-uncased'

es_val_input, es_val_masks = preprocess_bert(es_val, es_tokenizer)
eng_val_input, eng_val_masks = preprocess_bert(eng_val, eng_tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

##Make predictions

In [ ]:
#es_model1_pred = assign_class(es_model1.predict([es_val_input, es_val_masks]))
es_model2_pred = assign_class(es_model2.predict([es_val_input, es_val_masks]))
eng_model_pred = assign_class(eng_model.predict([eng_val_input, eng_val_masks]))
eng_model_pred2 = assign_class(eng_model2.predict([eng_val_input, eng_val_masks]))

In [ ]:
es_model2_pred

array([[0.0263709 ],
       [0.05390123],
       [0.00559049],
       ...,
       [0.01828228],
       [0.04691067],
       [0.00566016]], dtype=float32)

##Function to determine majority

In [ ]:
def majority_vote(pred1, pred2, pred3):
  pred =[]
  for i in range(len(pred1)):
    sum = pred1[i] + pred2[i] + pred3[i]
    if(sum >= 2):
      pred.append(1)
    else:
      pred.append(0)
  return pred

##Confusion Matrix and Classification Report for Majority Vote

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

y_true = labels
y_pred = majority_vote(es_model1_pred, eng_model_pred, eng_model_pred2)

print(confusion_matrix(y_true, y_pred, labels=[1,0]))
print(classification_report(y_true, y_pred, target_names=["non-prof", "prof"]))
print('f1-score:', f1_score(y_true, y_pred))

[[ 410   67]
 [  46 1477]]
              precision    recall  f1-score   support

    non-prof       0.96      0.97      0.96      1523
        prof       0.90      0.86      0.88       477

    accuracy                           0.94      2000
   macro avg       0.93      0.91      0.92      2000
weighted avg       0.94      0.94      0.94      2000

f1-score: 0.8788853161843516


#Combine models for Classification Task : Maximum Probability Try

##Import data

In [ ]:
#Import data in English and in Spanish
#Import data for English and Spanish tweets
eng_val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val-eng.csv')
es_val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val.csv')

#use dataframe to assign appropriate values to variables
eng_val = eng_val_data.tweet
es_val = es_val_data.tweet

labels = eng_val_data.label

In [14]:
#Import data in English and in Spanish
#Import data for English and Spanish tweets
test_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/test.csv')

#use dataframe to assign appropriate values to variables
test = test_data.tweet


In [15]:
len(test)

27000

##Preprocess data using bert using bert-base and using mbert

In [ ]:
eng_tokenizer = 'bert-base-uncased'
es_tokenizer = 'bert-base-multilingual-uncased'
max_len = 80

es_val_input, es_val_masks = preprocess_bert(es_val, es_tokenizer, max_len)
eng_val_input, eng_val_masks = preprocess_bert(eng_val, eng_tokenizer, max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [16]:
es_tokenizer = 'bert-base-multilingual-uncased'
max_len = 80

es_test_input, es_test_masks = preprocess_bert(test, es_tokenizer, max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


##Compile and load weights of es and eng model

In [17]:
#define variable for model creation
es_pt_model = 'bert-base-multilingual-uncased'
eng_pt_model = 'bert-base-uncased'
num_labels = 1
loss = 'binary_crossentropy'
max_len = 80
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

#Compile models
aug_model = create_transformer_model(es_pt_model, num_labels, max_len)
es_model = create_transformer_model(es_pt_model, num_labels, max_len)

aug_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
es_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

#we load both models (if needed)
es_model.load_weights('/content/drive/MyDrive/Colab/saved_model/mBert_profner3.hdf5')
aug_model.load_weights('/content/drive/MyDrive/Colab/saved_model/mBert_profner_aug.hdf5')
#eng_model2.load_weights('/content/drive/MyDrive/Colab/saved_model/engBert_profner2.hdf5')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
train_input (InputLayer)        [(None, 80)]         0                                            
__________________________________________________________________________________________________
train_masks (InputLayer)        [(None, 80)]         0                                            
__________________________________________________________________________________________________
tf_bert_for_sequence_classifica TFSequenceClassifier 167357185   train_input[0][0]                
                                                                 train_masks[0][0]                
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1)            2           tf_bert_for_sequence_classi

##Predict probability of each class

In [18]:
es_prob = es_model.predict([es_test_input, es_test_masks])
aug_prob = aug_model.predict([es_test_input, es_test_masks])

In [ ]:
print(es_prob[:10])
print(aug_prob[:10])
print(labels[:10])

In [19]:
def pick_max_prob(es_prob, aug_prob):
  pred = []
  for i in range(len(es_prob)):
    pred.append(np.max([es_prob[i], aug_prob[i]]))

  return pred

In [ ]:
###Idea to try: pick positive class from weighted class and negative class from other model (if there is a disagreement)
'''
es_pred = assign_class(es_prob)
weight_pred = assign_class(aug_prob)
pred = []
for i in range(len(es_prob)):
  if es_pred[i] != weight_pred[i]:
    if es_pred[i] == 0:
      pred.append(0)
    else:
      pred.append(1)
  else:
    if es_pred[i] == 0:
      pred.append(0)
    else:
      pred.append(1)
'''

##Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

#y_true = labels
y_pred = assign_class(pick_max_prob(es_prob, aug_prob))

print(confusion_matrix(y_true, y_pred, labels=[1,0]))
print(classification_report(y_true, y_pred, target_names=["non-prof", "prof"]))
print('f1-score:', f1_score(y_true, y_pred))

NameError: ignored

In [21]:
len(y_pred)

27000

#Concatenate transformers idea

Step one:
  encode Spanish text to mbert, encode English text to bert-base.
  Encodings will be used to init a transoformer model in Spanish, and the init a transformer model in English. 
  

In [ ]:
!pip install transformers
from transformers import BertTokenizer, TFBertForSequenceClassification

In [44]:
#import data in English and Spanish for training
eng_train_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/train-eng.csv')
es_train_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/train.csv')

eng_train = eng_train_data.tweet
es_train = es_train_data.tweet

labels = eng_train_data.label


In [45]:
#shuffle data for train data in english
idx = np.random.permutation(len(eng_train))
eng_x_train, eng_y_train = eng_train[idx],labels[idx]

#shuffle train data in Spanish
#idx = np.random.permutation(len(es_train))
es_x_train, es_y_train = es_train[idx],labels[idx]

In [46]:
#Import data in English and in Spanish Validation
#Import data for English and Spanish tweets
eng_val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val-eng.csv')
es_val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val.csv')

#use dataframe to assign appropriate values to variables
eng_val = eng_val_data.tweet
es_val = es_val_data.tweet

val_labels = eng_val_data.label

In [47]:
#Import data in English and in Spanish Validation
#Import data for English and Spanish tweets
es_test_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/test.csv')

#use dataframe to assign appropriate values to variables
es_test = es_test_data.tweet

In [48]:
#Encode test data into bert eng and m bert
eng_tokenizer = 'bert-base-uncased'
es_tokenizer = 'bert-base-multilingual-uncased'
max_len = 80

#train
eng_test_input, eng_test_masks = preprocess_bert(es_test, eng_tokenizer, max_len)
es_test_input, es_test_masks = preprocess_bert(es_test, es_tokenizer, max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [56]:
eng_tokenizer = 'bert-base-uncased'
es_tokenizer = 'bert-base-multilingual-uncased'
max_len = 80

#train
#eng_train_input, eng_train_masks = preprocess_bert(eng_x_train, eng_tokenizer, max_len)
#es_train_input, es_train_masks = preprocess_bert(es_x_train, es_tokenizer, max_len)

#validation
es_val_input, es_val_masks = preprocess_bert(es_val, es_tokenizer, max_len)
eng_val_input, eng_val_masks = preprocess_bert(eng_val, eng_tokenizer, max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [57]:
#make labels into numpy arrays, and prep x_val
x_val = [eng_val_input, eng_val_masks, es_val_input, es_val_masks]

#turn labels into numpy arrays and then concatenate
y_train = np.asarray(eng_y_train, dtype='int32')
#es_y_train = np.asarray(es_y_train, dtype='int32')
#y_train = np.append(eng_y_train, es_y_train)

#turn labels into numpy arrays and then concatenate (the same labels in same order bc not randomized)
y_val = np.asarray(val_labels, dtype='int32')
#y_val = np.append(val_labels, val_labels)

In [ ]:
print(type(y_train))
print(y_train.shape)
print(y_train.dtype)
print(y_train[:100])

<class 'numpy.ndarray'>
(6000,)
int32
[0 1 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1
 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]


##Create model with two transformers

In [51]:
import tensorflow as tf

#making labels into numpy arrays for training in transformers and keras
def create_concat_transformer_model(pre_trained_model_1, pre_trained_model_2, num_labels, max_len):
  eng_transformer_model = TFBertForSequenceClassification.from_pretrained(pre_trained_model_1, num_labels=num_labels)
  es_transformer_model = TFBertForSequenceClassification.from_pretrained(pre_trained_model_2, num_labels=num_labels)
  
  #We create keras tensors english
  eng_input_ids = tf.keras.layers.Input(shape=(max_len,), name='eng_train_input', dtype=tf.int32)
  eng_input_masks = tf.keras.layers.Input(shape=(max_len), name='eng_train_masks', dtype=tf.int32)

  #We create keras tensors Spanish
  es_input_ids = tf.keras.layers.Input(shape=(max_len,), name='es_train_input', dtype=tf.int32)
  es_input_masks = tf.keras.layers.Input(shape=(max_len), name='es_train_masks', dtype=tf.int32)
  
  #take into account single dimension
  eng_seq_outputs = eng_transformer_model(eng_input_ids, eng_input_masks)[0]
  es_seq_outputs = es_transformer_model(es_input_ids, es_input_masks)[0]

  #concatenate outputs sequences from transformer models
  concat_outputs = tf.concat([eng_seq_outputs, es_seq_outputs],1)
  outputs = tf.keras.layers.Dense(num_labels, activation='sigmoid')(concat_outputs)
  
  model = tf.keras.models.Model(inputs=[eng_input_ids,eng_input_masks, es_input_ids, es_input_masks], outputs=outputs)

  print(model.summary())

  return model

In [52]:
#call create model function and compile
eng_trans_model = 'bert-base-uncased'
es_trans_model = 'bert-base-multilingual-uncased'
num_labels = 1
loss = 'binary_crossentropy'
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)


concat_model = create_concat_transformer_model(eng_trans_model, es_trans_model, num_labels, 80)
concat_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
eng_train_input (InputLayer)    [(None, 80)]         0                                            
__________________________________________________________________________________________________
eng_train_masks (InputLayer)    [(None, 80)]         0                                            
__________________________________________________________________________________________________
es_train_input (InputLayer)     [(None, 80)]         0                                            
__________________________________________________________________________________________________
es_train_masks (InputLayer)     [(None, 80)]         0                                            
____________________________________________________________________________________________

In [ ]:
#fit model to input data
class_weights = {0:1.,
                 1:30.}
with tf.device('/device:GPU:0'):
  concat_model.fit([eng_train_input, eng_train_masks, es_train_input, es_train_masks], y_train, validation_data=(x_val, y_val), epochs=4, batch_size = 32)

Epoch 1/4
188/188 [==============================] - 155s 717ms/step - loss: 0.4250 - accuracy: 0.8262 - val_loss: 0.1693 - val_accuracy: 0.9450
Epoch 2/4
188/188 [==============================] - 133s 706ms/step - loss: 0.1299 - accuracy: 0.9578 - val_loss: 0.1554 - val_accuracy: 0.9520
Epoch 3/4
188/188 [==============================] - 133s 706ms/step - loss: 0.0602 - accuracy: 0.9823 - val_loss: 0.1733 - val_accuracy: 0.9410
Epoch 4/4
188/188 [==============================] - 133s 706ms/step - loss: 0.0339 - accuracy: 0.9900 - val_loss: 0.1734 - val_accuracy: 0.9545


In [ ]:
#save model
#concat_model.save_weights('/content/drive/MyDrive/Colab/saved_model/concat_model_weight2.hdf5')

##Predict and Confusion Matrix

In [53]:
concat_model.load_weights('/content/drive/MyDrive/Colab/saved_model/concat_model.hdf5')

In [58]:
pred = concat_model.predict(x_val)

In [59]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

y_true = y_val
y_pred = assign_class(pred)

print(confusion_matrix(y_true, y_pred, labels=[1,0]))
print(classification_report(y_true, y_pred, target_names=["non-prof", "prof"]))
print('f1-score:', f1_score(y_true, y_pred))

[[ 433   44]
 [  50 1473]]
              precision    recall  f1-score   support

    non-prof       0.97      0.97      0.97      1523
        prof       0.90      0.91      0.90       477

    accuracy                           0.95      2000
   macro avg       0.93      0.94      0.94      2000
weighted avg       0.95      0.95      0.95      2000

f1-score: 0.9020833333333333


# Concatenate with extra transformer layer
We concatenate both transformer output, as above, but we feed it to a keras transformer layer and then to the dense layer. 

Hypothesis is that it will be the best performing


In [ ]:
!pip install transformers
from transformers import BertTokenizer, TFBertForSequenceClassification

##Import, preprocess and tokenize data

In [ ]:
#import data in English and Spanish for training
eng_train_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/train-eng.csv')
es_train_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/train.csv')

eng_train = eng_train_data.tweet
es_train = es_train_data.tweet

labels = eng_train_data.label


In [ ]:
#shuffle data for train data in english
idx = np.random.permutation(len(eng_train))
eng_x_train, eng_y_train = eng_train[idx],labels[idx]

#shuffle train data in Spanish
#idx = np.random.permutation(len(es_train))
es_x_train, es_y_train = es_train[idx],labels[idx]

In [ ]:
#Import data in English and in Spanish Validation
#Import data for English and Spanish tweets
eng_val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val-eng.csv')
es_val_data = pd.read_csv('/content/drive/MyDrive/Datasets/profner/val.csv')

#use dataframe to assign appropriate values to variables
eng_val = eng_val_data.tweet
es_val = es_val_data.tweet

val_labels = eng_val_data.label

In [ ]:
eng_tokenizer = 'bert-base-uncased'
es_tokenizer = 'bert-base-multilingual-uncased'
max_len = 80

#train
eng_train_input, eng_train_masks = preprocess_bert(eng_x_train, eng_tokenizer, max_len)
es_train_input, es_train_masks = preprocess_bert(es_x_train, es_tokenizer, max_len)

#validation
es_val_input, es_val_masks = preprocess_bert(es_val, es_tokenizer, max_len)
eng_val_input, eng_val_masks = preprocess_bert(eng_val, eng_tokenizer, max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [ ]:
#make labels into numpy arrays, and prep x_val
x_val = [eng_val_input, eng_val_masks, es_val_input, es_val_masks]

#turn labels into numpy arrays and then concatenate
y_train = np.asarray(eng_y_train, dtype='int32')

#turn labels into numpy arrays and then concatenate (the same labels in same order bc not randomized)
y_val = np.asarray(val_labels, dtype='int32')


##Create model with keras transformer layer

In [ ]:
!pip install keras-transformer
import keras_transformer

  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp37-none-any.whl size=12942 sha256=a6133288ce4ba1efee17e3e9bc89212619b4644ed820457beb8dc3a13ec49e1f
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp37-none-any.whl size=7554 sha256=358d71c600ea569a49347b294fd77315a06914d7721c72b9b4964a2ad5d7e877
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp37-none-any.whl size=15611 sha256=e10eb2ff5573a999becb5b312ab8d3013d778a2d495414abdf5bc045118fbdf7
  Stored in directory: /root/.cache/pip/wheels/b5/b4/49/0a0c27dcb93c13af02fea254ff51d1a43a924dd4e5b7a7164d
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.14.0-cp37-none-any.whl size=5269 sha256=717821a839c54ad7f5c49b6ed0a06535d9b2540084

In [ ]:
#transformer_block code from keras
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [ ]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [ ]:
import tensorflow as tf
from keras_transformer import get_model

#making labels into numpy arrays for training in transformers and keras
def create_transformer_model(pre_trained_model_1, pre_trained_model_2, num_labels, max_len):
  #bert models
  eng_transformer_model = TFBertForSequenceClassification.from_pretrained(pre_trained_model_1, num_labels=num_labels)
  es_transformer_model = TFBertForSequenceClassification.from_pretrained(pre_trained_model_2, num_labels=num_labels)
  
  print('bert done')
  #transoformer_block using keras implementation
  transoformer_block = TransformerBlock(768 * 2, num_heads=2, ff_dim=256)
  #embedding tokens and positioning
  embedding_layer = TokenAndPositionEmbedding(max_len, )
  print('transformer done')
  #We create keras tensors english
  eng_input_ids = tf.keras.layers.Input(shape=(max_len,), name='eng_train_input', dtype=tf.int32)
  eng_input_masks = tf.keras.layers.Input(shape=(max_len), name='eng_train_masks', dtype=tf.int32)
  print('input english done')
  #We create keras tensors Spanish
  es_input_ids = tf.keras.layers.Input(shape=(max_len,), name='es_train_input', dtype=tf.int32)
  es_input_masks = tf.keras.layers.Input(shape=(max_len), name='es_train_masks', dtype=tf.int32)
  print('input español done')

  #take into account single dimension
  eng_seq_outputs = eng_transformer_model(eng_input_ids, eng_input_masks)[0]
  es_seq_outputs = es_transformer_model(es_input_ids, es_input_masks)[0]
  print('outputs done')

  #concatenate outputs sequences from transformer models
  concat_outputs = tf.concat([eng_seq_outputs, es_seq_outputs],1)
  print('concat_done')
  trans_layer = transoformer_block(concat_outputs)
  print('transfomer done')
  trans_layer = tf.keras.layers.GlobalAveragePooling1D()(trans_layer)
  print('avg pooling')
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(trans_layer)
  print('outputs')
  
  model = tf.keras.models.Model(inputs=[eng_input_ids,eng_input_masks, es_input_ids, es_input_masks], outputs=outputs)

  print(model.summary())

  return model

In [ ]:
#call create model function and compile
eng_trans_model = 'bert-base-uncased'
es_trans_model = 'bert-base-multilingual-uncased'
num_labels = 1
loss = 'binary_crossentropy'
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)


concat_model = create_transformer_model(eng_trans_model, es_trans_model, num_labels, 80)
concat_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert done
transformer done
input english done
input español done
outputs done
concat_done


StagingError: ignored

In [ ]:
#fit model to input data
with tf.device('/device:GPU:0'):
  concat_model.fit([eng_train_input, eng_train_masks, es_train_input, es_train_masks], y_train, validation_data=(x_val, y_val), epochs=7, batch_size = 32)

Epoch 1/7
188/188 [==============================] - 164s 728ms/step - loss: 0.5325 - accuracy: 0.7853 - val_loss: 0.2151 - val_accuracy: 0.9370
Epoch 2/7
188/188 [==============================] - 134s 713ms/step - loss: 0.2138 - accuracy: 0.9373 - val_loss: 0.1906 - val_accuracy: 0.9430
Epoch 3/7
188/188 [==============================] - 134s 712ms/step - loss: 0.1244 - accuracy: 0.9688 - val_loss: 0.2083 - val_accuracy: 0.9395
Epoch 4/7
188/188 [==============================] - 134s 711ms/step - loss: 0.1054 - accuracy: 0.9730 - val_loss: 0.1985 - val_accuracy: 0.9410
Epoch 5/7
188/188 [==============================] - 134s 712ms/step - loss: 0.0806 - accuracy: 0.9801 - val_loss: 0.2301 - val_accuracy: 0.9335
Epoch 6/7
188/188 [==============================] - 134s 711ms/step - loss: 0.0817 - accuracy: 0.9789 - val_loss: 0.1994 - val_accuracy: 0.9425
Epoch 7/7
188/188 [==============================] - 134s 712ms/step - loss: 0.0546 - accuracy: 0.9868 - val_loss: 0.2491 - val_ac

In [ ]:
#save model
concat_model.save_weights('/content/drive/MyDrive/Colab/saved_model/transformer_model.hdf5')

##Predict and Confusion Matrix

In [ ]:
pred = concat_model.predict(x_val)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

y_true = y_val
y_pred = assign_class(pred)

print(confusion_matrix(y_true, y_pred, labels=[1,0]))
print(classification_report(y_true, y_pred, target_names=["non-prof", "prof"]))
print('f1-score:', f1_score(y_true, y_pred))

[[ 416   61]
 [  51 1472]]
              precision    recall  f1-score   support

    non-prof       0.96      0.97      0.96      1523
        prof       0.89      0.87      0.88       477

    accuracy                           0.94      2000
   macro avg       0.93      0.92      0.92      2000
weighted avg       0.94      0.94      0.94      2000

f1-score: 0.8813559322033897


#Prep submission results in tsv file format

In [50]:
test_data.head()

,Unnamed: 0,tweet
0,1242399368143147012,Cuidados de tu mascota durante la epidemia del...
1,1242402119623286791,#ULTIMAHORA: Los contagios en España por #coro...
2,1242402388574601216,"Y para que veáis que nos quejamos de vicio, pa..."
3,1242402392475340803,Amancio Ortega propuesto como Premio Princesa ...
4,1242402598642167808,⚠️ #covid19 #Almería Las tiendas de barrio en ...


In [51]:
test_data.rename(columns={'Unnamed: 0': 'tweet_id'}, inplace=True)
test_data.tweet_id[:5]

0    1242399368143147012
1    1242402119623286791
2    1242402388574601216
3    1242402392475340803
4    1242402598642167808
Name: tweet_id, dtype: int64

In [52]:
results = pd.DataFrame({
    'tweet_id': test_data.tweet_id,
    'label': es_pred
})
results.head()

,tweet_id,label
0,1242399368143147012,0
1,1242402119623286791,0
2,1242402388574601216,1
3,1242402392475340803,1
4,1242402598642167808,0


In [53]:
results.to_csv('/content/drive/MyDrive/profner_results/test_results_weight.tsv', sep='\t')